In [ ]:
import pandas as pd
from surprise import Dataset, Reader, SVD, accuracy
import pickle

In [ ]:
ratings_file_path = 'data/ratings.csv'
ratings = pd.read_csv(ratings_file_path)

In [ ]:
reader = Reader(rating_scale=(0.5, 5.0))

In [ ]:
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

In [ ]:
model = SVD()
trainset = data.build_full_trainset()
model.fit(trainset)

In [ ]:
predictions = model.test(trainset.build_testset())

In [ ]:
rmse = accuracy.rmse(predictions)
print("RMSE del modelo en el conjunto de entrenamiento:", rmse)

In [ ]:
model_file_path = 'svd_model.pkl'
with open(model_file_path, 'wb') as model_file:
    pickle.dump(model, model_file)